# GPT-3.5-Turbo on GSM8K

In [40]:
%pip install openai==0.28
%pip install datasets
%pip install tqdm
# %pip install -r ../MMLU/requirements.txt
# %pip install torch torchvision torchaudio
%pip install llmlingua

import openai
import re
import time

import numpy as np

from tqdm import tqdm
from datasets import load_dataset

  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached aiohttp-3.9.5-cp311-cp311-macosx_10_9_x86_64.whl.metadata (7.5 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-macosx_10_9_x86_64.whl.metadata (33 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.6.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached frozenlist-1.4.1-cp311-cp311-macosx_10_9_x86_64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp311-cp311-macosx_10_9_x86_64.whl.metadata (4.2 kB)
  Using cached yarl-1.9.4-cp311-cp311-macosx_10_9_x86_64.whl.metadata (31 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
Using cached reques

In [41]:
from llmlingua import PromptCompressor

llm_lingua = PromptCompressor(device_map="cpu")
# compressed_prompt = llm_lingua.compress_prompt(prompt, instruction="", question="", target_token=200)


# > {'compressed_prompt': 'Question: Sam bought a dozen boxes, each with 30 highlighter pens inside, for $10 each box. He reanged five of boxes into packages of sixlters each and sold them $3 per. He sold the rest theters separately at the of three pens $2. How much did make in total, dollars?\nLets think step step\nSam bought 1 boxes x00 oflters.\nHe bought 12 * 300ters in total\nSam then took 5 boxes 6ters0ters.\nHe sold these boxes for 5 *5\nAfterelling these  boxes there were 3030 highlighters remaining.\nThese form 330 / 3 = 110 groups of three pens.\nHe sold each of these groups for $2 each, so made 110 * 2 = $220 from them.\nIn total, then, he earned $220 + $15 = $235.\nSince his original cost was $120, he earned $235 - $120 = $115 in profit.\nThe answer is 115',
#  'origin_tokens': 2365,
#  'compressed_tokens': 211,
#  'ratio': '11.2x',
#  'saving': ', Saving $0.1 in GPT-4.'}

## Or use the phi-2 model,
# llm_lingua = PromptCompressor("microsoft/phi-2")

## Or use the quantation model, like TheBloke/Llama-2-7b-Chat-GPTQ, only need <8GB GPU memory.
## Before that, you need to pip install optimum auto-gptq
# llm_lingua = PromptCompressor("TheBloke/Llama-2-7b-Chat-GPTQ", model_config={"revision": "main"})

Loading checkpoint shards: 100%|██████████| 2/2 [00:25<00:00, 12.54s/it]


In [42]:
openai.api_key = "sk-"

In [43]:
gsm8k = load_dataset('gsm8k', 'main')
validation_index = np.load('../gsm8k/lib_prompt/validation_index.npy')
validation_data = gsm8k['train'].select(validation_index)
gsm8k_test = gsm8k['test']

In [44]:
gsm8k['train'][0]['question']

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [45]:
gsm8k_test = gsm8k['test']

In [46]:
prompt_complex = open('../gsm8k/lib_prompt/prompt_hardest.txt').read()

In [47]:
print(prompt_complex)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [48]:
%pip install tenacity

from tenacity import (
    retry,
    stop_after_attempt,
    wait_chain,
    wait_fixed
) 

@retry(wait=wait_chain(*[wait_fixed(3) for i in range(3)] +
                       [wait_fixed(5) for i in range(2)] +
                       [wait_fixed(10)]))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

Note: you may need to restart the kernel to use updated packages.


In [49]:
def test_answer(pred_str, ans_str):
    pattern = '\d*\.?\d+'
    pred = re.findall(pattern, pred_str)
    if(len(pred) >= 1):
        # print(pred_str)
        pred = pred[-1]
        gold = re.findall(pattern, ans_str)
        # print(ans_str)
        gold = gold[-1]
        return pred == gold
    else: return False

def parse_pred_ans(filename):
    with open(filename) as fd: lines = fd.readlines()
    am, a = None, None
    num_q, acc = 0, 0
    current_mode = 'none'
    questions = []
    ans_pred = []
    ans_gold = []
    for l in lines:
        if(l.startswith('Q: ')):
            if(am is not None and a is not None):
                questions.append(q)
                ans_pred.append(am)
                ans_gold.append(a)
                if(test_answer(am, a)):
                    acc += 1
            current_mode = 'q'
            q = l
            num_q += 1
        elif(l.startswith('A_model:')):
            current_mode = 'am'
            am = l
        elif(l.startswith('A:')):
            current_mode = 'a'
            a = l
        else:
            if(current_mode == 'q'): q += l
            elif(current_mode == 'am'): am += l
            elif(current_mode == 'a'): a += l
            else:
                raise ValueError(current_mode)
                
    questions.append(q)
    ans_pred.append(am)
    ans_gold.append(a)
    if(test_answer(am, a)):
        acc += 1
    print('num_q %d correct %d ratio %.4f' % (num_q, acc, float(acc / num_q)))
    return questions, ans_pred, ans_gold

def test_finished(ans_model):
    if('answer is' in ans_model): return True
    else: return False

def extract_ans(ans_model):
    ans_model = ans_model.split('\n')
    ans = []
    residual = []
    for li, al in enumerate(ans_model):
        ans.append(al)
        if('answer is' in al):
            break
    residual = list(ans_model[li + 1:])
    ans = '\n'.join(ans)
    residual = '\n'.join(residual)
    return ans, residual

In [50]:
prompt_q = prompt_complex + '\nQuestion: ' + gsm8k_test[1]['question'] + '\n'

In [51]:
print(prompt_q)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [52]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {"role": "user", "content": prompt_q},
    ]
)

In [53]:
response['choices'][0]['message']['content']

'It takes 2 bolts of blue fiber and half that much white fiber, which means it takes 2 + (1/2)*2 = 2 + 1 = 3 bolts of fiber in total. \n\nTherefore, it takes 3 bolts in total.'

In [54]:
print(response['choices'][0]['message']['content'])

It takes 2 bolts of blue fiber and half that much white fiber, which means it takes 2 + (1/2)*2 = 2 + 1 = 3 bolts of fiber in total. 

Therefore, it takes 3 bolts in total.


# Complex Prompt Random Sampling, Acc 77.1

In [55]:
custom_size = len(gsm8k_test['question'])//20

i = 0
with open('outputs/test_gpt_3.5_turbo_complex.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'][:custom_size], gsm8k_test['answer'][:custom_size]), 
                               total=len(gsm8k_test['question'][:custom_size])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'  
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ]
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

  0%|          | 0/65 [00:00<?, ?it/s]

100%|██████████| 65/65 [03:24<00:00,  3.15s/it]


In [20]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex.txt')


num_q 65 correct 45 ratio 0.6923


# Complex Prompt Random Sampling, 압축

In [60]:
custom_size = len(gsm8k_test['question'])//20

i = 0
with open('outputs/test_gpt_3.5_turbo_complex_압축.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'][:custom_size], gsm8k_test['answer'][:custom_size]), 
                               total=len(gsm8k_test['question'][:custom_size])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'  

        comped_prompt_q = llm_lingua.compress_prompt(prompt_q, target_token=300)

        print(comped_prompt_q)
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": comped_prompt_q},
                ]
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

  0%|          | 0/65 [12:34<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex_압축.txt')

num_q 65 correct 45 ratio 0.6923


# Complex Prompt Greedy Decoding, Acc 78.85

In [35]:
i = 0
with open('outputs/test_gpt_3.5_turbo_complex_temp_0.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'  
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ],
                temperature=0
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1319/1319 [2:04:18<00:00,  5.65s/it]


In [36]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex_temp_0.txt')

num_q 1319 correct 1040 ratio 0.7885


# Baseline Prompt Greedy Decoding, Acc 74.98

In [8]:
prompt_original = open('../gsm8k/lib_prompt/prompt_original.txt').read()
print(prompt_original)

In [12]:
i = 0
with open('outputs/test_gpt_3.5_turbo_original_temp_0.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_original + '\nQuestion: ' + q + '\n'  
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ],
                temperature=0
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1319/1319 [2:36:17<00:00,  7.11s/it]


In [13]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_original_temp_0.txt')

num_q 1319 correct 989 ratio 0.7498


# Baseline Prompt, Dialog In-Context Learning, Acc 76.8

In [24]:
def make_dialog_prompt(prompt):
    messages = []
    messages.append({"role": "system", "content": "Follow the given examples and answer the question."})
    cases = prompt.split("\n\n")
    for c in cases[:-1]:
        question = c.split("\n")[:2]
        messages.append({"role": "user", "content": "\n".join(question)})
        answer = c.split("\n")[2:]
        messages.append({"role": "assistant", "content": "\n".join(answer)})
    messages.append({"role": "user", "content": cases[-1] + "Let's think step by step"})
    return messages

In [30]:
i = 0
with open('outputs/test_gpt_3.5_turbo_original_dialog_icl.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_original + '\nQuestion: ' + q + '\n'
        dialog_prompt = make_dialog_prompt(prompt_q)
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=dialog_prompt,
              temperature=0
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1319/1319 [2:57:13<00:00,  8.06s/it]


In [31]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_original_dialog_icl.txt')

num_q 1319 correct 1013 ratio 0.7680


# Complex Prompt, Dialog In-Context Learning, Acc 

In [34]:
i = 0
with open('outputs/test_gpt_3.5_turbo_complex_dialog_icl.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'
        dialog_prompt = make_dialog_prompt(prompt_q)
        
        response = completion_with_backoff(
              model="gpt-3.5-turbo",
              messages=dialog_prompt,
              temperature=0
            )
        ans_model = response['choices'][0]['message']['content']
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1319/1319 [2:43:01<00:00,  7.42s/it]


In [35]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex_dialog_icl.txt')

num_q 1319 correct 988 ratio 0.7491
